<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_Upside_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytz


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\rathi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

print_date_time()

gen_output = 0

Run date time (IST): 2025-02-20 20:31:00


In [3]:
# install libraries
!pip install yfinance
!pip install pyxirr
#!pip install talib-binary


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\rathi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\rathi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
# import libraries

#%matplotlib inline
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import datetime
#from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
#import pandas_datareader.data as web
import yfinance as yfin
#import talib
#yfin.pdr_override()
#import pandas_ta as ta

#from google.colab import data_table

In [5]:
# set start and end date
start_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [ ]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/'

mypfs = pd.read_csv(path_mypf + 'myPortfolioStocks.csv')
mypps = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

df_mypf = pd.merge(mypfs, mypps, how='right', on="Symbol")

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_mymmtm_df():
  mymmtm_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myProspects-Momentum.csv')
  return mymmtm_df

def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'ADANITRANS', 'SREERAYALK', 'SCHAEFFLER ', 'EQUITAS', 'UJJIVAN', 'TIPSINDLTD', 'SYMPHONY']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

mymmtm_df = get_mymmtm_df()

df_mypf = df_mypf[df_mypf['InPortfolio'].isnull()]
#cols = ['Symbol', 'Criteria', 'Strategy', 'LatestQtr', 'Target','Breakout','StarStock', 'Conviction', 'Category', 'Stock', 'MedianPE']
#df_mypf = df_mypf[cols]
my_prospects = get_stock_ids(df_mypf)
df_stocks = df_mypf[df_mypf['Symbol'].isin(my_prospects)]

In [7]:
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

In [8]:
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'TFrame', 'LatestQtr', 'StarStock', 'BizFinTech', 'Conviction', 'Category']
df_stocks = df_stocks[common_cols]

In [9]:

# function to get prospects technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)

    stock_df['Min'] = round(min(stock_df['Close']),2)
    #stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
    #stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    #stock_df['Close'] = round(stock_df['Close'],0)
    #stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to get common features for prospects
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in stock_ids:
      #print(stock_id)
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)

  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, mymmtm_df, on= 'Symbol', how='left')
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  return df_prec_dev

stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

In [10]:
df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

In [11]:
# prospects stocks

def get_prospects_features(df_common_features):

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  #tmp_df = df_common_features[df_common_features['Stock'].isin(df_mypf[df_mypf['InPortfolio'] != 'NA'].Stock.values)]
  tmp_df = df_common_features
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Today%'] = round((tmp_df['Close'] - tmp_df['Prev_Close'])*100/tmp_df['Prev_Close'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']

  return tmp_df

df_prospects_features = get_prospects_features(df_common_features)

qualified stocks: 170
with latest results: 161
still star stocks: 89


In [12]:
cols = ['Symbol', 'Close', 'FTT',  'Dev%_200', 'Dev%_PE',  'Today%', 'FTT%', 'Gained%', 'Criteria', 'Strategy','LatestQtr', 'StarStock', 'Conviction', 'Category']
df_prospects_features = df_prospects_features[cols]

if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_prospects_features.to_csv('/content/drive/My Drive/data/stocks/myStocks-Portfolio-Analysis.csv', index=False)

In [13]:
# xy24, xy25
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['XY24', 'XY25'])].sort_values(by = 'FTT%', ascending=False)
df_tmp

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Today%,FTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
29,BOROLTD,322.35,500.5,-18.39,NaN,-1.30,55.27,7.34,XY24,ATH,1,0,L,GLASS
6,AMBUJACEM,482.85,733.0,-18.71,27.70,0.61,51.81,0.61,XY24,BTT,0,0,M,CEMENT
133,ROSSARI,636.55,965.0,-21.02,NaN,0.24,51.60,1.45,XY24,NTT,1,1,M,CHEMICALS
158,UTIAMC,971.85,1395.0,-14.60,-26.32,2.40,43.54,22.79,XY24,ATH,1,1,M,FINANCE
26,BIOCON,336.00,397.0,-4.00,-42.67,-0.24,18.15,36.91,XY24,NTT,1,1,M,PHARMA
91,JSWHL,15580.45,7487.0,45.37,129.81,2.44,-51.95,158.00,XY24,BTT,1,0,H,FINANCE


In [14]:
# x40
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['X40'])].sort_values(by = 'FTT%', ascending=False)
df_tmp.head(10)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Today%,FTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
165,WHIRLPOOL,987.35,2270.00,-46.80,-47.53,-1.60,129.91,0.45,X40,NTT,1,0,M,DURABLES
124,PFIZER,4110.05,6318.30,-19.88,-19.49,-0.50,53.73,0.65,X40,ATH,1,0,H,PHARMA
112,NAM-INDIA,536.30,805.90,-18.00,-20.69,2.76,50.27,26.86,X40,ATH,1,0,M,FINANCE
17,BAJAJ-AUTO,8630.20,12666.40,-10.97,61.42,1.78,46.77,10.00,X40,ATH,1,1,X,AUTO
134,SANOFI,5168.60,7552.00,-22.93,22.86,-1.21,46.11,0.37,X40,NTT,0,0,H,PHARMA
76,ICICIPRULI,574.80,791.05,-14.34,12.10,-0.21,37.62,11.01,X40,ATH,1,0,H,INSURANCE
4,AKZOINDIA,3289.45,4433.10,-1.45,1.71,-0.12,34.77,47.96,X40,ATH,1,1,M,PAINTS
75,ICICIGI,1727.30,2268.71,-8.99,-15.12,-0.10,31.34,13.17,X40,ATH,1,1,H,INSURANCE
125,PGHH,13987.35,18062.58,-11.26,-13.18,0.67,29.14,2.35,X40,ATH,0,0,X,FMCG
87,ITC,402.05,514.63,-12.41,10.76,-1.07,28.00,3.05,X40,ATH,1,1,H,FMCG


In [15]:
# x40n
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['X40N'])].sort_values(by = 'FTT%', ascending=False)
df_tmp.head(10)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Today%,FTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
141,SUNTV,579.30,1092.00,-21.08,-2.44,0.56,88.50,3.02,X40N,BTT,1,1,M,ENTERTAINMENT
52,EQUITASBNK,60.84,113.10,-22.00,NaN,0.05,85.90,1.94,X40N,ATH,1,0,M,BANKS
56,FINCABLES,930.25,1657.02,-28.33,28.81,0.18,78.13,11.54,X40N,ATH,1,1,M,CABLES
143,SYMPHONY,1144.60,2037.54,-14.90,NaN,2.81,78.01,40.56,X40N,ATH,1,1,M,DURABLES
149,TEAMLEASE,1985.50,3487.00,-31.77,-38.04,-1.93,75.62,0.00,X40N,NTT,1,0,M,MISC
110,MOTILALOFS,636.35,1000.00,-14.01,-4.11,3.79,57.15,83.12,X40N,NTT,1,1,H,FINANCE
119,OFSS,8679.80,12945.45,-17.71,71.49,-1.29,49.14,25.34,X40N,ATH,1,1,H,IT
0,3MINDIA,27442.80,40020.60,-17.99,-32.47,1.64,45.83,2.87,X40N,ATH,1,1,H,MISC
162,VINATIORGA,1605.45,2325.26,-13.64,-13.58,1.93,44.84,9.65,X40N,ATH,1,0,X,CHEMICALS
24,BAYERCROP,4749.25,6805.00,-19.78,2.31,0.29,43.29,8.70,X40N,NTT,1,0,H,CHEMICALS


In [16]:
# x200, xr and ar, exp50
df_tmp = df_prospects_features[~df_prospects_features['Criteria'].isin(['X40','X40N'])].sort_values(by = 'FTT%', ascending=False)
df_tmp = df_tmp[df_tmp['Criteria'].notnull()]
df_tmp.head(10)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Today%,FTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
19,BAJAJHIND,22.25,442.93,-36.43,NaN,-0.76,1890.70,5.45,XR,ATH,1,0,L,SUGAR
5,ALOKINDS,17.06,79.81,-28.92,NaN,1.31,367.82,5.70,AR,ATH,1,0,L,TEXTILES
36,CENTRALBK,47.37,190.06,-18.33,-24.34,2.33,301.22,3.41,XR,ATH,1,0,L,BANKS
7,APEX,215.25,849.16,-11.42,NaN,-2.31,294.50,13.20,AR,ATH,1,0,L,MISC
78,IGL,203.61,707.00,-9.51,-60.32,5.01,247.23,30.77,AR,BTT,1,1,M,GAS
48,DREAMFOLKS,277.75,813.19,-36.88,NaN,1.46,192.78,1.46,EXP50,ATH,1,1,M,MISC
127,PNB,95.65,232.89,-13.05,-54.18,3.00,143.48,4.13,XR,ATH,1,1,L,BANKS
83,INFIBEAM,20.48,49.37,-26.86,NaN,0.64,141.06,9.87,AR,ATH,1,1,L,IT
144,TASTYBITE,8936.75,20387.67,-19.55,NaN,0.01,128.13,4.70,XR,ATH,1,0,M,FMCG
61,GLAND,1519.95,3442.00,-14.80,-10.40,1.36,126.45,5.54,XR,NTT,1,0,H,PHARMA


In [17]:
# all prospects
df_tmp = df_prospects_features.sort_values(by = 'FTT%', ascending=False)
df_tmp.head(10)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Today%,FTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
168,YESBANK,18.21,383.47,-17.23,-38.34,1.17,2005.82,6.37,NaN,NaN,1,0,L,BANKS
19,BAJAJHIND,22.25,442.93,-36.43,NaN,-0.76,1890.70,5.45,XR,ATH,1,0,L,SUGAR
116,NETWORK18,47.67,676.90,-38.88,-132.57,1.58,1319.97,4.45,NaN,NaN,1,0,L,MEDIA
142,SUZLON,54.88,415.22,-12.89,-29.22,1.33,656.60,51.18,NaN,NaN,1,0,L,ELECTRICAL
169,ZEEL,100.56,583.44,-23.82,-8.37,1.56,480.19,4.13,NaN,NaN,1,0,M,ENTERTAINMENT
5,ALOKINDS,17.06,79.81,-28.92,NaN,1.31,367.82,5.70,AR,ATH,1,0,L,TEXTILES
73,IBREALEST,133.73,619.91,2.08,NaN,4.68,363.55,34.81,NaN,NaN,0,0,L,REALTY
36,CENTRALBK,47.37,190.06,-18.33,-24.34,2.33,301.22,3.41,XR,ATH,1,0,L,BANKS
7,APEX,215.25,849.16,-11.42,NaN,-2.31,294.50,13.20,AR,ATH,1,0,L,MISC
78,IGL,203.61,707.00,-9.51,-60.32,5.01,247.23,30.77,AR,BTT,1,1,M,GAS


In [18]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 68.0 seconds
